# Clustering Crypto

In [30]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [31]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [32]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
#a = requests.get(url)
#data = a.json()["Data"]

#crypto_df_test = pd.DataFrame(data = data)

#crypto_df_test = crypto_df_test.T
#crypto_df_test = crypto_df_test[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]

#crypto_df_test


In [33]:
# Alternatively, use the provided csv file:
from pathlib import Path

crypto_data_path = Path('../Starter_Files/Resources/crypto_data.csv')
crypto_df = pd.read_csv(crypto_data_path)
crypto_df.index = crypto_df["Unnamed: 0"]
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,,
42,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [34]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]


In [35]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.drop(crypto_df[crypto_df["IsTrading"] == False].index)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610


In [36]:
# Keep only cryptocurrencies with a working algorithm
# assuming all listed algos work

In [37]:
# Remove the "IsTrading" column
crypto_df.drop("IsTrading", axis=1, inplace=True)

In [38]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [39]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.drop(crypto_df[crypto_df["TotalCoinsMined"] == 0].index)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [40]:
# Drop rows where there are 'N/A' text values
#crypto_df.loc[crypto_df["TotalCoinSupply"] == 'N/A']
# none found using loc on each column
crypto_df.loc[~crypto_df.index.duplicated(), :]

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [41]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame()
coins_name.index= crypto_df.index
coins_name

""
Unnamed: 0
42
404
808
1337
BTC
...
ZEPH
GAP
BDX


In [42]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.reset_index(drop=True, inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
2,808,SHA-256,PoW/PoS,0.000000e+00,0
3,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
4,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [43]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df)

In [44]:
# Standardize data
scaler = StandardScaler().fit_transform(X)
print(scaler[0:5])

[[-0.10282804 -0.03823596 -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]
 [-0.07661326 -0.03823596 -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]
 [-0.10282804 -0.03823596 -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]
 [ 0.62458365 -0.03823596 -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]
 [-0.10238266 -0.03823596 -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]]


### Reducing Dimensions Using PCA

In [45]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(scaler)


In [46]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["pc1", "pc2", "pc3"]
)
pcs_df.index = coins_name.index
pcs_df

,pc1,pc2,pc3
Unnamed: 0,,,
42,-0.244914,0.047588,-0.202258
404,-0.257017,0.243568,-0.098475
808,-0.115201,-0.037157,-0.151726
1337,0.274522,-0.167886,0.210733
BTC,0.002267,-0.210312,-0.077309
...,...,...,...
ZEPH,2.591337,-0.902053,-0.498429
GAP,-0.261590,0.092784,-0.099334
BDX,0.219394,-0.259848,-0.021666


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [47]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


c:\Users\gunin\anaconda3\envs\pyvizenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [50]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.DataFrame()
concat_lst = [clustered_df, crypto_df, pcs_df]
clustered_df = pd.concat(concat_lst, axis=1)
clustered_df["CoinName"] = coins_name.index
clustered_df["class"] = model.labels_

clustered_df

# Output unexpectedly started giving errors, gave the correct output on first few runs of the code. Not sure where i went wrong


ValueError: Length of values (685) does not match length of index (1370)

### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot.scatter(x= "TotalCoinsMined", y= "TotalCoinSupply", hover_cols=["CoinName"])

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies
